In [12]:
import numpy as np
import os
from typing import Generator

In [13]:
cache_path_fstring = "cubes_{0}.npy"

def get_cache_raw(cache_path: str) -> list[np.ndarray]:
    """
    Loads a Cache File for a given pathname

    Parameters:
    cache_path (str): the file location to look for the cache file

    Returns:
    list[np.ndarray]: the list of polycubes from the cache

    """
    if os.path.exists(cache_path):

        polycubes = np.load(cache_path, allow_pickle=True)

        return polycubes
    else:
        return None


def get_cache(n: int) -> np.ndarray:
    """
    Loads a Cache File for a given size of polycube

    Parameters:
    n (int): the size of polycube to load the cache of

    Returns:
    list[np.ndarray]: the list of polycubes of that size from the cache

    """
    cache_path = cache_path_fstring.format(n)
    print(f"\rLoading polycubes n={n} from cache: ", end="")
    polycubes = get_cache_raw(cache_path)
    print(f"{len(polycubes)} shapes")
    return polycubes

In [14]:
data = get_cache(8)

Loading polycubes n=8 from cache: 6922 shapes


In [15]:

def all_rotations(polycube: np.ndarray) -> Generator[np.ndarray, None, None]:
    """
    Calculates all rotations of a polycube.

    Adapted from https://stackoverflow.com/questions/33190042/how-to-calculate-all-24-rotations-of-3d-array.
    This function computes all 24 rotations around each of the axis x,y,z. It uses numpy operations to do this, to avoid unecessary copies.
    The function returns a generator, to avoid computing all rotations if they are not needed.

    Parameters:
    polycube (np.array): 3D Numpy byte array where 1 values indicate polycube positions

    Returns:
    generator(np.array): Yields new rotations of this cube about all axes

    """
    def single_axis_rotation(polycube, axes):
        """Yield four rotations of the given 3d array in the plane spanned by the given axes.
        For example, a rotation in axes (0,1) is a rotation around axis 2"""
        for i in range(4):
            yield np.rot90(polycube, i, axes)

    # 4 rotations about axis 0
    yield from single_axis_rotation(polycube, (1, 2))

    # rotate 180 about axis 1, 4 rotations about axis 0
    yield from single_axis_rotation(np.rot90(polycube, 2, axes=(0, 2)), (1, 2))

    # rotate 90 or 270 about axis 1, 8 rotations about axis 2
    yield from single_axis_rotation(np.rot90(polycube, axes=(0, 2)), (0, 1))
    yield from single_axis_rotation(np.rot90(polycube, -1, axes=(0, 2)), (0, 1))

    # rotate about axis 2, 8 rotations about axis 1
    yield from single_axis_rotation(np.rot90(polycube, axes=(0, 1)), (0, 2))
    yield from single_axis_rotation(np.rot90(polycube, -1, axes=(0, 1)), (0, 2))

In [16]:
def pack2d(polycube: np.ndarray) -> bytes:
    """
    Converts a 3D ndarray into a single bytes object that unique identifies 
    the polycube, is hashable, comparable, and allows to reconstruct the 
    original polycube ndarray.

    Parameters:
    polycube (np.array): 3D Numpy byte array where 1 values indicate polycube positions,
        and 0 values indicate empty space. Must be of type np.int8.

    Returns:
    cube_id (bytes): a bytes representation of the polycube

    """
    data =  polycube.shape[0].to_bytes(1, 'little') \
            + polycube.shape[1].to_bytes(1, 'little') \
            + np.packbits(polycube.flatten(), bitorder='little').tobytes()
    return data


def unpack2d(cube_id: bytes) -> np.ndarray:
    """
    Converts a bytes object back into a 3D ndarray

    Parameters:
    cube_id (bytes): a unique bytes object

    Returns:
    polycube (np.array): 3D Numpy byte array where 1 values indicate 
        cube positions
        
    """
    # Extract shape information
    shape = (cube_id[0], cube_id[1])
    size = shape[0] * shape[1]
    polycube = np.unpackbits(np.frombuffer(cube_id[2:], dtype=np.uint8), count=size, bitorder='little').reshape(shape)
    return polycube

In [17]:
def view_2d(polycube):
    view1 = np.sum(polycube, axis=0)
    view2 = np.sum(polycube, axis=1)
    view3 = np.sum(polycube, axis=2)
    
    views = [pack2d(view1), pack2d(view2), pack2d(view3)]
    views = sorted(views)
    return views[0] + views[1] + views[2]

In [20]:
import math
import matplotlib.pyplot as plt
def render_shapes(shapes: list[np.ndarray], path: str):
    n = len(shapes)
    dim = max(max(a.shape) for a in shapes)
    i = math.isqrt(n) + 1
    voxel_dim = dim * i
    voxel_array = np.zeros((voxel_dim + i, voxel_dim + i, dim), dtype=np.byte)
    pad = 1
    for idx, shape in enumerate(shapes):
        x = (idx % i) * dim + (idx % i)
        y = (idx // i) * dim + (idx // i)
        xpad = x * pad
        ypad = y * pad
        s = shape.shape
        voxel_array[x:x + s[0], y:y + s[1], 0: s[2]] = shape

    # voxel_array = crop_cube(voxel_array)
    colors = np.empty(voxel_array.shape, dtype=object)
    colors[:] = '#FFD65DC0'

    ax = plt.figure(figsize=(20, 16), dpi=600).add_subplot(projection='3d')
    ax.voxels(voxel_array, facecolors=colors, edgecolor='k', linewidth=0.1)

    ax.set_xlim([0, voxel_array.shape[0]])
    ax.set_ylim([0, voxel_array.shape[1]])
    ax.set_zlim([0, voxel_array.shape[2]])
    plt.axis("off")
    ax.set_box_aspect((1, 1, voxel_array.shape[2] / voxel_array.shape[0]))
    plt.savefig(path + ".png", bbox_inches='tight', pad_inches=0)


In [22]:
seen_views = {}
for cube in data:
    views = set()
    for rotation in all_rotations(cube):
        views.add(view_2d(rotation))
    for view in views:
        if(view in seen_views.keys()):
            render_shapes([cube], 'Cube A')
            render_shapes([seen_views[view]], 'Cube B')
            raise Exception(f"found duplicate view {unpack2d(view)}\n for cubes {cube}\n and {seen_views[view]}\n")
    for view in views:
        seen_views[view] = cube

Exception: found duplicate view [[1 1 1]
 [1 0 0]
 [1 0 0]]
 for cubes [[[1 1 1]
  [0 1 0]
  [0 1 0]]

 [[0 0 0]
  [0 0 0]
  [0 1 0]]

 [[0 0 0]
  [0 0 0]
  [1 1 0]]]
 and [[[1 1 1]
  [0 1 0]
  [0 1 0]]

 [[0 0 0]
  [0 0 0]
  [0 1 0]]

 [[0 0 0]
  [0 0 0]
  [0 1 1]]]
